In [24]:
import pandas as pd
import numpy as np
import datetime
import argparse
import shutil
import sys
import os

from docplex.mp.model import Model

In [25]:
#CAMBIAR CUANDO SE QUIERA CORRER DESDE ACA O CUANDO SE QUIERA DEESCARGAR UNA NUEVA VERSION PARA CORRER AUTOMATICAMENTE

#ap = argparse.ArgumentParser()
#ap.add_argument("-m", "--manual", required=False, help="Indicar si es corrida manual corta (1) o larga (2)", default= 0)
#args = vars(ap.parse_args())

args = {'manual': '1'}

## Paths